In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import numpy.matlib

In [ ]:
path_base = "INSERT_PATH_TO_YOUR_RUN_DIRECTORY"
file_FE_0 = path_base + '/output/FE_CANOPY.0'
file_lad = path_base + '/initial/LAD_profile.csv'
file_FE_0_lad = path_base + '/initial/FE_CANOPY.0'
hc_ref = 30.0 # specify canopy height

In [ ]:
lad_data = pd.read_csv(file_lad)
lad_data

In [ ]:
fe_data = xr.open_dataset(file_FE_0)
fe_data

In [ ]:
zPos_1d = fe_data.zPos.isel(time=0,yIndex=0,xIndex=0).values

In [ ]:
prof_lad_v = lad_data['LAD']
prof_z_v = lad_data['z/hc']*hc_ref

In [ ]:
z_max = np.max(prof_z_v)
print('z_max=',z_max)
z_diff = np.abs(zPos_1d-z_max)
ind_FE = np.where(z_diff==np.min(z_diff))[0][0]
print('ind_FE=',ind_FE)
print('zPos_1d[ind_FE]=',zPos_1d[ind_FE])

In [ ]:
LAD_p1 = np.interp(zPos_1d[0:ind_FE+1], prof_z_v, prof_lad_v)
print('LAD_p1.shape=',LAD_p1.shape)

In [ ]:
LAD_v = np.zeros(zPos_1d.shape)
LAD_v[0:ind_FE+1] = LAD_p1
print('LAD_v=',LAD_v)

In [ ]:
LAD_2d = np.tile(LAD_v, [fe_data.sizes['xIndex'], 1])
print('LAD_2d.shape=',LAD_2d.shape)
LAD_3d = np.tile(LAD_2d, [fe_data.sizes['yIndex'], 1, 1])
print('LAD_3d.shape=',LAD_3d.shape)
LAD_3d_n = np.transpose(LAD_3d,[2,0,1])
print('LAD_3d_n.shape=',LAD_3d_n.shape)

In [ ]:
ds_out = fe_data

In [ ]:
ds_out['CanopyLAD'][0,:,:,:] = LAD_3d_n

In [ ]:
ds_out.to_netcdf(file_FE_0_lad)